# Approximation Theory and Applications

## Chapter 4 - Algebraic Polynomials 

Numerical experiments with Chebshev polynomial approximation schemes.


### Runge's Phenomenon

We consider the function $f : [-1, 1] \to \mathbb{R}$, 
$$
   f(x) = \frac{1}{1 + 25 x^2}
$$
Note that $f$ is analytic on $[-1,1]$, hence from our work on trigonometric approximation we expect excellent approximation properties. We choose a uniform grid, 
$$
  x_j = -1 + 2j/N, \qquad j = 0, \dots, N
$$
and interpolate $f$ at those grid points. 

In [ ]:
include("tools.jl")

In [ ]:
f = x -> 1/(1+25*x^2)
NN1 = [5, 8, 10]
NN2 =  5:5:30

# do not do this - we will learn later how to do polynomial interpolation 
# in a numerically stable way.
function naive_poly_fit(f, N)
   X = range(-1, 1, length=N+1)
   A = [ X[m+1]^n for m = 0:N, n = 0:N ]
   return A \ f.(X)
end

# don't do this either, this is just a quick naive code suitable for the 
# current experiment. 
naive_poly_eval(x, c) = sum( c[n] * x^(n-1) for n = 1:length(c) )
	
# first plot 
xp = range(-1, 1, length=300)
P1 = plot(xp, f.(xp); lw=4, label = "exact",
          size = (400, 400), xlabel = L"x")
for (iN, N) in enumerate(NN1)
   xi = [(-1 + 2*m/N) for m = 0:N]
   c = naive_poly_fit(f, N)
   plot!(P1, xp, naive_poly_eval.(xp, Ref(c)), c = iN+1, lw=2,label = L"p_{%$(N)}")
   plot!(P1, xi, f.(xi), lw=0, c = iN+1, m = :o, ms=3, label = "")
end 

# second plot 
xerr = range(-1, 1, length=3_000)
err = [ norm( f.(xerr) - naive_poly_eval.(xerr, Ref(naive_poly_fit(f, N))), Inf )
      for N in NN2 ]
P2 = plot(NN2, err, lw = 3, label = L"\Vert f - I_N f \Vert", 
         yscale = :log10, xlabel = L"N", legend = :topleft)
plot(P1, P2, size = (600, 300), title = "Witch of Agnesi")


Don't trust any package unless you know what they are using behind the scenes!!! More often than not, packages will use this naive polynomial interpolation as the default.

We now know of course that Chebyshev nodes are the way to go. And while we are at it, we will also switch to the Chebyshev basis. As a reminder, the Chebushev basis is given by the recursion
$$
    T_0(x) = 1, \quad T_1(x) = x, \quad T_{n+1}(x) = 2 x T_n(x) - T_{n-1}(x).
$$
and the Chebyshev nodes via 
$$
    x_j = \cos(\pi j/N)
$$

In [ ]:
function chebbasis(x, N)
   T = zeros(N+1)
   T[1] = 1 
   T[2] = x 
   for n = 2:N
      T[n+1] = 2 * x * T[n] - T[n-1] 
   end 
   return T 
end

chebnodes(N) = [ cos( π * n / N ) for n = N:-1:0 ]

function chebinterp(f, N)
   X = chebnodes(N)
   A = zeros(N+1, N+1)
   for (ix, x) in enumerate(X)
      A[ix, :] .= chebbasis(x, N)
   end
   return A \ f.(X)
end

chebeval(x, c) = dot(c, chebbasis(x, length(c)-1))


In [ ]:
xp = range(-1, 1, length=300)
P1 = plot(xp, f.(xp); lw=4, label = "exact",
          size = (400, 400), xlabel = L"x")
for (iN, N) in enumerate(NN1)
   xi = [(-1 + 2*m/N) for m = 0:N]
   c = chebinterp(f, N)
   plot!(P1, xp, chebeval.(xp, Ref(c)), c = iN+1, lw=2,label = L"p_{%$(N)}")
   plot!(P1, xi, f.(xi), lw=0, c = iN+1, m = :o, ms=3, label = "")
end 

# second plot 
xerr = range(-1, 1, length=3_000)
err = [ norm( f.(xerr) - chebeval.(xerr, Ref(chebinterp(f, N))), Inf )
      for N in NN2 ]
P2 = plot(NN2, err, lw = 3, label = L"\Vert f - I_N f \Vert", 
         yscale = :log10, xlabel = L"N", legend = :topleft)
plot!(P2, NN2[3:end], 4*(1.23).^(-NN2[3:end]), c=:black, ls=:dash, label = L"\rho^{-N}")
plot(P1, P2, size = (600, 300), title = "Witch of Agnesi")



## The Fast Chebyshev Transform

Because of the intimate connection between Chebyshev polynomials and trigonometric polynomials we can use the FFT to implement the interpolation operator.

In [ ]:
"""
Fast and stable implementation based on the FFT. This uses 
the connection between Chebyshev and trigonometric interpolation.
But this transform needs the reverse chebyshev nodes.
"""
chebinterp(f, N) = fct(f.(reverse(chebnodes(N))))

using FFTW 

function fct(A::AbstractVector)
    N = length(A)
    F = real.(ifft([A[1:N]; A[N-1:-1:2]]))
   return [[F[1]]; 2*F[2:(N-1)]; [F[N]]]
end


"""
Evaluate a polynomial with coefficients F̃ in the Chebyshev basis. 
This avoids storing the basis and is significantly faster.
"""
function chebeval(x, F̃) 
    T0 = one(x); T1 = x 
    p = F̃[1] * T0 + F̃[2] * T1 
    for n = 3:length(F̃)
        T0, T1 = T1, 2*x*T1 - T0 
        p += F̃[n] * T1 
    end 
    return p 
end 


In [ ]:
xp = range(-1, 1, length=300)
P1 = plot(xp, f.(xp); lw=4, label = "exact",
          size = (400, 400), xlabel = L"x")
for (iN, N) in enumerate(NN1)
   xi = [(-1 + 2*m/N) for m = 0:N]
   c = chebinterp(f, N)
   plot!(P1, xp, chebeval.(xp, Ref(c)), c = iN+1, lw=2,label = L"p_{%$(N)}")
   plot!(P1, xi, f.(xi), lw=0, c = iN+1, m = :o, ms=3, label = "")
end 

# second plot 
xerr = range(-1, 1, length=3_000)
err = [ norm( f.(xerr) - chebeval.(xerr, Ref(chebinterp(f, N))), Inf )
      for N in NN2 ]
P2 = plot(NN2, err, lw = 3, label = L"\Vert f - I_N f \Vert", 
         yscale = :log10, xlabel = L"N", legend = :topleft)
plot!(P2, NN2[3:end], 4*(1.23).^(-NN2[3:end]), c=:black, ls=:dash, label = L"\rho^{-N}")
plot(P1, P2, size = (600, 300), title = "Witch of Agnesi")



In [ ]:
# let's take this to the extreme ... 
f = x -> 1 / (1 + 1000 * x^2)
NN = 10:10:1_400 

xerr = range(-1, 1, length=3_000)
err = [ norm( f.(xerr) - chebeval.(xerr, Ref(chebinterp(f, N))), Inf )
        for N in NN ]
P2 = plot(NN, err, lw = 3, label = L"\Vert f - I_N f \Vert", 
         yscale = :log10, xlabel = L"N", legend = :topright, 
         yticks = [1.0, 1e-3, 1e-6, 1e-9, 1e-12, 1e-15])
plot!(NN, 0*NN .+ eps(), c=:red, ls = :dot, label = "eps" )
tt = [250, 750]
plot!(tt, 4*(1+1/sqrt(1000)).^(-tt), c=:black, ls=:dash, label = L"\rho^{-N}")

### Barycentric Interpolation 

In [ ]:
function chebbary(x, f, N)
    p = q = 0.0 
    for j = 0:N
        xj = cos(π*j/N)
        fj = f(xj)
        λj = (-1)^j * 0.5 * (1 + (1 <= j < N))
        p += fj * λj / (x - xj)
        q += λj / (x - xj)
    end
    return p/q
end

In [ ]:
f = x -> 1 / (1 + 25 * x^2)
xp = range(-1+0.123*1e-6, 1-0.123*1e-6, length=500)
plot(xp, chebbary.(xp, f, 100), ylims=[-0.5, 1.5] )

In [ ]:

f = x -> 1 / (1 + 1000 * x^2)
NN = 10:10:1200

xerr = range(-1+0.123*1e-6, 1-0.123*1e-6, length=3_000)
err = [ norm( f.(xerr) - chebbary.(xerr, f, N), Inf )
        for N in NN ]
P2 = plot(NN, err, lw = 3, label = L"\Vert f - I_N f \Vert", 
         yscale = :log10, xlabel = L"N", legend = :topright, 
         yticks = [1.0, 1e-3, 1e-6, 1e-9, 1e-12, 1e-15])
plot!(NN, 0*NN .+ eps(), c=:red, ls = :dot, label = "eps" )
tt = [250, 750]
plot!(tt, 4*(1+1/sqrt(1000)).^(-tt), c=:black, ls=:dash, label = L"\rho^{-N}")